<a href="https://colab.research.google.com/github/nayvti380/Transfer_Learning/blob/main/CatsxDogs_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning - Problema cats x dogs

Implementação de rede convolucional usando transfer learning para diferenciação das categorias gato e cachorro.

 O banco de dados original se encontra [aqui](https://www.kaggle.com/c/dogs-vs-cats). Dentre essas imagens foram separadas 8000 imagens para treinamento e 2000 imagens para teste.



## Procedimentos Iniciais

Apagar a pasta catsxdogs caso algum novo dado seja incluído na pasta:

In [ ]:
#!rm -rf catsxdogs

Download da pasta:

In [ ]:
!git clone https://github.com/cunhamaicon/catsxdogs

Cloning into 'catsxdogs'...
remote: Enumerating objects: 10055, done.
remote: Total 10055 (delta 0), reused 0 (delta 0), pack-reused 10055 (from 1)
Receiving objects: 100% (10055/10055), 301.00 MiB | 10.28 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Updating files: 100% (10018/10018), done.


Importação dos pacotes:

In [ ]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout

## CNN - Transfer learning:

> Adicionar aspas



Importando o modelo MobileNet que foi previamente treinado no ImageNet e descartando a última camada de neurônios:

In [ ]:
model=MobileNet(weights='imagenet',include_top=False)

<ipython-input-4-43901aabfbda>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model=MobileNet(weights='imagenet',include_top=False)


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Criando a saída do modelo MobileNet:

In [ ]:
x=model.output
x=GlobalAveragePooling2D()(x)

Adicionando uma camada intermediária e a camada final:

In [ ]:
x=Dense(50,activation='relu')(x)
preds=Dense(1,activation='sigmoid')(x)
model=Model(inputs=model.input,outputs=preds)

Visualizando todas as camadas da nova rede criada usando o modelo MobileNetV2:

In [ ]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_layer
1 conv1
2 conv1_bn
3 conv1_relu
4 conv_dw_1
5 conv_dw_1_bn
6 conv_dw_1_relu
7 conv_pw_1
8 conv_pw_1_bn
9 conv_pw_1_relu
10 conv_pad_2
11 conv_dw_2
12 conv_dw_2_bn
13 conv_dw_2_relu
14 conv_pw_2
15 conv_pw_2_bn
16 conv_pw_2_relu
17 conv_dw_3
18 conv_dw_3_bn
19 conv_dw_3_relu
20 conv_pw_3
21 conv_pw_3_bn
22 conv_pw_3_relu
23 conv_pad_4
24 conv_dw_4
25 conv_dw_4_bn
26 conv_dw_4_relu
27 conv_pw_4
28 conv_pw_4_bn
29 conv_pw_4_relu
30 conv_dw_5
31 conv_dw_5_bn
32 conv_dw_5_relu
33 conv_pw_5
34 conv_pw_5_bn
35 conv_pw_5_relu
36 conv_pad_6
37 conv_dw_6
38 conv_dw_6_bn
39 conv_dw_6_relu
40 conv_pw_6
41 conv_pw_6_bn
42 conv_pw_6_relu
43 conv_dw_7
44 conv_dw_7_bn
45 conv_dw_7_relu
46 conv_pw_7
47 conv_pw_7_bn
48 conv_pw_7_relu
49 conv_dw_8
50 conv_dw_8_bn
51 conv_dw_8_relu
52 conv_pw_8
53 conv_pw_8_bn
54 conv_pw_8_relu
55 conv_dw_9
56 conv_dw_9_bn
57 conv_dw_9_relu
58 conv_pw_9
59 conv_pw_9_bn
60 conv_pw_9_relu
61 conv_dw_10
62 conv_dw_10_bn
63 conv_dw_10_relu
64 conv_pw_10
65 conv_

Definindo qual camada da rede será treinada. Nesse caso somente as duas últimas camadas adicionadas:

In [ ]:
for layer in model.layers[:88]:
    layer.trainable=False
for layer in model.layers[88:]:
    layer.trainable=True

## ImageDataGenerator

Definindo o tamanho de cada batch:

In [ ]:
batch_size = 32

Cada imagem do banco será apresentada a rede de uma forma diferente através do ImageDataGenerator:

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.4,
                                   zoom_range = 0.4,
                                   height_shift_range=0.3,
                                   width_shift_range=0.3,
                                   rotation_range=50,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('catsxdogs/training_set',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('catsxdogs/test_set',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Treinamento

Definindo os parâmetros de compilação da rede:

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


Fazendo o treinamento da rede:

In [ ]:
history = model.fit(training_set,
                   steps_per_epoch=int(8000/batch_size),
                   epochs=10,
                   validation_data=test_set,
                   validation_steps=int(2000/batch_size))


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 122s 447ms/step - accuracy: 0.5121 - loss: 0.8243 - val_accuracy: 0.5368 - val_loss: 0.7326
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 103s 414ms/step - accuracy: 0.5178 - loss: 0.7502 - val_accuracy: 0.5958 - val_loss: 0.6668
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 142s 568ms/step - accuracy: 0.5664 - loss: 0.6932 - val_accuracy: 0.6633 - val_loss: 0.6241
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 103s 414ms/step - accuracy: 0.5996 - loss: 0.6641 - val_accuracy: 0.7051 - val_loss: 0.5909
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 103s 412ms/step - accuracy: 0.6294 - loss: 0.6369 - val_accuracy: 0.7399 - val_loss: 0.5630
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 102s 408ms/step - accuracy: 0.6481 - loss: 0.6208 - val_accuracy: 0.7676 - val_loss: 0.5391
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 103s 410ms/step - accuracy: 0.6821 - loss: 0.6009 - val_accuracy: 0.7923 - val_loss: 0.5171
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 102s 408ms/step - accuracy: 0.6896 -

Salvando o modelo para utilização futura:

In [ ]:
model.save('catsxdogs_mobilenet.h5')
from google.colab import files
files.download('catsxdogs_mobilenet.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Previsão

Mostrando os arquivos da pasta single_prediction com imagens inéditas para a rede classificar:

In [ ]:
ls catsxdogs/single_prediction

cat_or_dog_1.jpg  cat_or_dog_2.jpg  chino1.jpg  floyd1.jpg  floyd2.jpg  floyd3.jpg  floyd4.jpg


Escolhendo uma imagem da pasta single_prediction para fazer a previsão:

In [ ]:
test_image = image.load_img('catsxdogs/single_prediction/cat_or_dog_1.jpg', target_size = (224, 224))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image/255

In [ ]:
result = model.predict(test_image)

if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
dog
